In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from nerfstudio.utils.eval_utils import eval_setup
# from ns_extension.utils.grouping import GroupingClassifier
import collab_splats

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Taichi] version 1.7.4, llvm 15.0.4, commit b4b956fd, linux, python 3.10.18


[I 07/31/25 18:57:23.155 7691] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


### Load configuration

In [ ]:
# Path to the config for a trained model
load_config = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/config.yml'
# load_config = '/workspace/fieldwork-data/birds/2024-02-06/environment/C0043/rade-features/2025-07-25_040743/config.yml'
load_config = Path(load_config)

config, pipeline, checkpoint_path, step = eval_setup(load_config)

### Question is whether to build grouping on top of the existing data?

In [ ]:
from PIL import Image
import numpy as np

from collab_splats.utils.features import resize_image
from collab_splats.utils.segmentation import Segmentation

segmentation = Segmentation(
    backend='mobilesamv2',
    strategy='object',
    device='cuda',
)
segmentation.strategy = 'auto'

### Start making our functions here

In [2]:
from collab_splats.utils.grouping import GroupingClassifier, GroupingParams

### Load a classifier

In [3]:
# Path to the config for a trained model
load_config = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/config.yml'
load_config = Path(load_config)

grouping_params = GroupingParams(segmentation_backend='mobilesamv2', segmentation_strategy='object', front_percentage=0.2, iou_threshold=0.1, num_patches=32)
grouping_classifier = GroupingClassifier(load_config=load_config, params=grouping_params)

Loading NeRF pipeline and model...


[18:57:28] Auto image downscale factor of 2                                                 ]8;id=585361;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=797708;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[18:57:49] use color only optimization with sigmoid activation                                         ]8;id=189508;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=753362;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#213\213]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/nerfstudio_models/step-00002
9999.ckpt

Loading segmentation module...


Using cache found in /workspace/models/hub/RogerQi_MobileSAMV2_main


checkpoint_load_scucess


### Associate masks

In [4]:
grouping_classifier.associate()

Processing frames:   0%|          | 0/441 [00:00<?, ?it/s]

/workspace/collab-splats/collab_splats/utils/camera_utils.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale)).transpose(0, 1).cuda()
/workspace/collab-splats/collab_splats/utils/camera_utils.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale)).transpose(0, 1).cuda()

/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
0: 1024x576 18 objects, 132.1ms
S

Processing masks:   0%|          | 0/18 [00:00<?, ?it/s]

skipping cudagraphs for unknown reason
/workspace/collab-splats/collab_splats/utils/camera_utils.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale)).transpose(0, 1).cuda()

0: 1024x576 21 objects, 20.6ms
Speed: 3.2ms preprocess, 20.6ms inference, 1.6ms postprocess per image at shape (1, 3, 1024, 1024)


Processing masks:   0%|          | 0/18 [00:00<?, ?it/s]

skipping cudagraphs for unknown reason


AttributeError: 'GroupingClassifier' object has no attribute 'config'

In [ ]:
camera_image = batch["image"]

# Forward pass through model to get metadata
_ = model.get_outputs(camera=camera)

# Segment the image
patch_mask = create_patch_mask(camera_image)

# Annoyingly have to pass as a numpy array
_, results = segmentation.segment(camera_image.detach().cpu().numpy())

# Create composite mask
composite_mask = create_composite_mask(results)

In [ ]:
front_gaussians = classifier.select_front_gaussians(model.info, composite_mask, patch_mask)

In [ ]:
# To initialize the classifier, we first need to build the mask association
# This involves 
# 1) segmenting each camera view
# 2) associating the masks to the gaussians


def build_mask_association(model, cameras):
    for camera in tqdm(cameras):
        camera = camera.to(model.device)

        front_gaussians, _ = get_patch_front_gaussian_of_mask(model, camera)
        labels = assign_labels(front_gaussians)
        self._update_gaussian_idx_bank(labels, front_gaussians)

        if self.num_mask == 0:
            self.assigned_gaussians = torch.unique(torch.cat(front_gaussians))


In [ ]:
def _update_memory_bank(self, labels: torch.Tensor, mask_gaussians: list[torch.Tensor]):
    for label, gaussians in zip(labels, mask_gaussians):
        self.maintain_gaussian_idx_bank(label.item(), gaussians)

    if self.num_mask == 0:
        self.assigned_gaussians = torch.unique(torch.cat(front_gaussians))

def assign_labels(mask_gaussians: list[torch.Tensor]) -> torch.Tensor:
    """
    Assign a label to each set of front-facing Gaussians (i.e., masks),
    based on their overlap with previously observed (and labeled) Gaussians.

    Args:
        mask_gaussians (list of torch.Tensor): List of 1D tensors, each containing 
        the indices of Gaussians corresponding to one object mask in the current view.

    Returns:
        torch.Tensor: Tensor of shape (num_masks,) containing the assigned label
        index for each input mask.
    """
    num_masks = len(mask_gaussians)

    # If this is the first view (no masks have been assigned yet), 
    # assign each mask a unique label (0 to num_masks - 1).
    if self.total_masks == 0:
        return torch.arange(num_masks, dtype=torch.long)

    # Otherwise, initialize an empty label tensor.
    labels = torch.zeros(num_masks, dtype=torch.long)

    # Loop over each mask's Gaussians in the current view
    for i, gaussians in enumerate(mask_gaussians):
        overlaps = []
        n_gaussians = len(gaussians)

        # Compare with each mask's bank of Gaussians seen so far
        for bank in self.memory_bank:
            union = torch.unique(torch.cat([bank, gaussians]))
            intersection = len(bank) + n_gaussians - len(union)

            # Compute IOCUR: intersection over (intersection + current size)
            io_cur = intersection / (n_gaussians + intersection + 1e-8)
            overlaps.append(io_cur)

        # Convert to tensor on the correct device for computation
        overlaps = torch.tensor(overlaps, dtype=torch.float32, device=self.device)

        # Choose the best-matching label based on maximum IOCUR
        selected = torch.argmax(overlaps)

        # If no match passes threshold, assign a new label
        if overlaps[selected] < self.iou_threshold:
            selected = self.total_masks
            self.total_masks += 1 # Increment the number of masks to account for new label

        labels[i] = selected

    return labels